In [1]:
import pandas as pd
import tabulate

In [2]:
# The path to our CSV files
schoolscsv = "schools_complete.csv"
studentscsv = "students_complete.csv"

# Read our csvfiles data into pandas
schools_df = pd.read_csv(schoolscsv)
students_df = pd.read_csv(studentscsv)

In [3]:
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [4]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
schools_df = schools_df.rename(columns={"name": "School Name"})

In [6]:
students_df = students_df.rename(columns={"school": "School Name"})

In [7]:
school_district = pd.merge(
    students_df, schools_df, on="School Name", suffixes=("_stu", "_sch"))

In [27]:
school_district.head()

,Student ID,name,gender,grade,School Name,reading_score,math_score,School ID,type,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,1910635


In [26]:
del school_district["size"]

In [10]:
budget = schools_df["budget"].sum()
budget

24649428

In [11]:
studentscount = students_df["Student ID"].count()
studentscount

39170

In [12]:
schoolscount = schools_df["School ID"].count()
schoolscount

15

In [13]:
math_mean = students_df["math_score"].mean()
math_mean

78.98537145774827

In [14]:
reading_mean = students_df["reading_score"].mean()
reading_mean

81.87784018381414

In [15]:
district_math = students_df.loc[students_df["math_score"] >= 65,:]
students_df.loc[students_df["math_score"] >= 65,:]["math_score"].count()


33188

In [16]:
district_reading = students_df.loc[students_df["reading_score"] >= 65,:]["reading_score"].count()
students_df.loc[students_df["reading_score"] >= 65,:]["reading_score"].count()

37681

In [17]:
district_math.head()

,Student ID,name,gender,grade,School Name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69


In [18]:
district_math["math_score"].count()

33188

In [19]:
students_df["Student ID"].count()

39170

In [20]:
math_passing = round((district_math["math_score"].count()/students_df["Student ID"].count())*100,2)
math_passing

84.73

In [21]:
reading_passing = round((district_reading/students_df["Student ID"].count())*100,2)
reading_passing

96.2

In [22]:
overall_rate = (math_passing+reading_passing)/2
overall_rate

90.465

In [23]:
districtsummary_df = pd.DataFrame({
    "Total Schools": schoolscount,
    "Total Students": studentscount,
    "Total Budget": budget,
    "Average Math Score": math_mean,
    "Average Reading Score": reading_mean,
    "% Passing Math": math_passing,
    "% Passing Reading": reading_passing,
    "Overall Passing Rate": overall_rate
}, index=[0])
districtsummary_df
districtsummary_df["Total Budget"] = districtsummary_df["Total Budget"].map("${:,.2f}".format)
districtsummary_df["Total Students"] = districtsummary_df["Total Students"].map("{:,}".format)

In [24]:
reorg_districtsummary = districtsummary_df[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
reorg_districtsummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,84.73,96.2,90.465


In [25]:
print(tabulate.tabulate(reorg_districtsummary, tablefmt='grid', showindex=True))

+---+----+--------+----------------+---------+---------+-------+------+--------+
| 0 | 15 | 39,170 | $24,649,428.00 | 78.9854 | 81.8778 | 84.73 | 96.2 | 90.465 |
+---+----+--------+----------------+---------+---------+-------+------+--------+


In [100]:
schoolgrp = school_district.groupby(["School Name"],as_index=False)
#schoolgrp.reset_index()
#schoolgrp["reading_score","math_score"].mean()
type(schoolgrp)
#schoolgrp.columns
#schoolgrp1 = school_district.groupby(["School Name"]).agg({"math_score" >65: 'count'})




TypeError: '>' not supported between instances of 'str' and 'int'

In [88]:
school_mean = pd.merge(schools_df, schoolgrp.mean(), on=['School Name'], suffixes=('_Sch','_scgrp'))

In [90]:
del school_mean["reading_score_Sch"]

In [91]:
del school_mean["School ID_scgrp"]

In [92]:
del school_mean["budget_scgrp"]

In [93]:
del school_mean["Student ID"]

In [94]:
school_mean = school_mean.rename(columns={"size": "Total Students","budget_Sch": "Total Budget","reading_score_scgrp": "Average Reading Score","math_score": "Average Math Score"})
school_mean

,School ID_Sch,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score
0,0,Huang High School,District,2917,1910635,81.182722,76.629414
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201
6,6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432
8,8,Holden High School,Charter,427,248087,83.814988,83.803279
9,9,Pena High School,Charter,962,585858,84.044699,83.839917


In [130]:
school_mean["Per Student Budget"] = school_mean["Total Budget"]/school_mean["Total Students"]
school_mean

,School ID_Sch,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


In [132]:
del school_mean["School ID_Sch"]

In [133]:
school_mean1 = pd.DataFrame(school_mean.set_index('School Name'))
school_mean

,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


In [134]:
school_mean1

,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
School Name,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,655.0
Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
Holden High School,Charter,427,248087,83.814988,83.803279,581.0


In [135]:
pass_reading_df = students_df.loc[students_df["reading_score"] >= 65,:]
pass_reading_df.head()

,Student ID,name,gender,grade,School Name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [136]:
reading_passed_grp = pass_reading_df.groupby("School Name")
reading_passed_grp["Student ID"].count()

School Name
Bailey High School       4705
Cabrera High School      1858
Figueroa High School     2788
Ford High School         2571
Griffin High School      1468
Hernandez High School    4385
Holden High School        427
Huang High School        2756
Johnson High School      4498
Pena High School          962
Rodriguez High School    3784
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: Student ID, dtype: int64

In [137]:
reading_passed_grp["Student ID"].count()/school_mean1["Total Students"]

School Name
Bailey High School       0.945539
Cabrera High School      1.000000
Figueroa High School     0.945405
Ford High School         0.938664
Griffin High School      1.000000
Hernandez High School    0.946063
Holden High School       1.000000
Huang High School        0.944806
Johnson High School      0.944760
Pena High School         1.000000
Rodriguez High School    0.946237
Shelton High School      1.000000
Thomas High School       1.000000
Wilson High School       1.000000
Wright High School       1.000000
dtype: float64

In [138]:
pass_math_df = students_df.loc[students_df["math_score"] >= 65,:]
math_passed_grp = pass_math_df.groupby("School Name")
math_passed_grp["Student ID"].count()

School Name
Bailey High School       3877
Cabrera High School      1858
Figueroa High School     2276
Ford High School         2142
Griffin High School      1468
Hernandez High School    3603
Holden High School        427
Huang High School        2267
Johnson High School      3712
Pena High School          962
Rodriguez High School    3117
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: Student ID, dtype: int64

In [139]:
math_passed_grp["Student ID"].count()/school_mean1["Total Students"]

School Name
Bailey High School       0.779140
Cabrera High School      1.000000
Figueroa High School     0.771787
Ford High School         0.782037
Griffin High School      1.000000
Hernandez High School    0.777346
Holden High School       1.000000
Huang High School        0.777168
Johnson High School      0.779668
Pena High School         1.000000
Rodriguez High School    0.779445
Shelton High School      1.000000
Thomas High School       1.000000
Wilson High School       1.000000
Wright High School       1.000000
dtype: float64

In [144]:
school_mean1["% Passing Math"] = round((math_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100,2)
school_mean1["% Passing Reading"] = round((reading_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100,2)
school_mean1["% Overall Passing Rate"] = round((school_mean1["% Passing Math"] + school_mean1["% Passing Reading"])/2,2)
school_mean1 = school_mean1.rename(columns={"type": "School Type", "Total Budget": "Total School Budget"})
school_mean1.head()

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,655.0,77.72,94.48,86.10
Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0,77.18,94.54,85.86
Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0,100.00,100.00,100.00
Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0,77.73,94.61,86.17
Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,100.00,100.00,100.00


In [146]:
reorg_schoolsummary = school_mean1[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
reorg_schoolsummary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,77.72,94.48,86.10
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,77.18,94.54,85.86
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.00,100.00,100.00
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,77.73,94.61,86.17
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.00,100.00,100.00


In [151]:
reset_reorg = reorg_schoolsummary.reset_index()
reset_reorg1 = reset_reorg.sort_values("School Name")
reset_reorg1.set_index("School Name")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,77.91,94.55,86.23
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.00,100.00,100.00
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,77.18,94.54,85.86
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,78.20,93.87,86.04
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.00,100.00,100.00
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,77.73,94.61,86.17
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.00,100.00,100.00
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,77.72,94.48,86.10
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,77.97,94.48,86.22


In [196]:
reset_reorg2 = reset_reorg.sort_values("% Overall Passing Rate",ascending = False)
type(reset_reorg2.head(5))

pandas.core.frame.DataFrame

In [162]:
reset_reorg3 = reset_reorg.sort_values("% Overall Passing Rate")
reset_reorg3.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,77.18,94.54,85.86
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,78.20,93.87,86.04
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,77.72,94.48,86.10
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,77.73,94.61,86.17
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,77.97,94.48,86.22


In [205]:
avgscores = students_df.groupby(['School Name','grade'],as_index=False)
avgscores.mean()
avgmath_grp = avgmath["math_score"].mean()
avgmath_grp
avgmath_table =avgmath_grp.pivot('School Name','grade')
avgmath_table

math_score                                 
grade                       10th       11th       12th        9th
School Name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

In [201]:
avgmath_table[avgmath_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

math_score                                 
grade                        9th       10th       11th       12th
School Name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [203]:
avgread_grp = avgmath["reading_score"].mean()
avgread_grp
avgread_table =avgread_grp.pivot('School Name','grade')
avgread_table

reading_score                                 
grade                          10th       11th       12th        9th
School Name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

In [204]:
avgread_table[avgread_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

reading_score                                 
grade                           9th       10th       11th       12th
School Name                                                         
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [208]:
reorg_schoolsummary["Per Student Budget"].min()

578.0

In [209]:
reorg_schoolsummary["Per Student Budget"].max()

655.0

In [210]:
# Create bins in which to place values based upon TED Talk views
bins = [575,595,615,635,655]

# Create labels for these bins
group_labels = ["<$595", "$595-$615", "$615-$635", "$635-$655"]

perStudent_series = pd.cut(reorg_schoolsummary["Per Student Budget"], bins, labels=group_labels)
perStudent_series

School Name
Huang High School        $635-$655
Figueroa High School     $635-$655
Shelton High School      $595-$615
Hernandez High School    $635-$655
Griffin High School      $615-$635
Wilson High School           <$595
Cabrera High School          <$595
Bailey High School       $615-$635
Holden High School           <$595
Pena High School         $595-$615
Wright High School           <$595
Rodriguez High School    $635-$655
Johnson High School      $635-$655
Ford High School         $635-$655
Thomas High School       $635-$655
Name: Per Student Budget, dtype: category
Categories (4, object): [<$595 < $595-$615 < $615-$635 < $635-$655]

In [223]:
reorg_schoolsummary2 = reorg_schoolsummary
reorg_schoolsummary2["Spending ranges (Per Student)"] = perStudent_series
reorg_schoolsummary2

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending ranges (Per Student),School Size
School Name,,,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,77.72,94.48,86.10,$635-$655,Medium(1000-2500)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,77.18,94.54,85.86,$635-$655,Medium(1000-2500)
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.00,100.00,100.00,$595-$615,Medium(1000-2500)
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,77.73,94.61,86.17,$635-$655,Large(2500-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.00,100.00,100.00,$615-$635,Medium(1000-2500)
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.00,100.00,100.00,<$595,Medium(1000-2500)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.00,100.00,100.00,<$595,Medium(1000-2500)
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,77.91,94.55,86.23,$615-$635,Large(2500-5000)
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.00,100.00,100.00,<$595,Small(<1000)


In [218]:
perStudent_grp = reorg_schoolsummary2.groupby("Spending ranges (Per Student)")
#perStudent_grp.mean()
perStudent_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending ranges (Per Student),,,,,
<$595,83.455399,83.933814,100.000000,100.000000,100.000000
$595-$615,83.599686,83.885211,100.000000,100.000000,100.000000
$615-$635,80.199966,82.425360,88.955000,97.275000,93.115000
$635-$655,77.866721,81.368774,80.962857,95.228571,88.095714


In [219]:
reorg_schoolsummary["Total Students"].min()

427

In [220]:
reorg_schoolsummary["Total Students"].max()

4976

In [221]:
# Create bins in which to place values based upon TED Talk views
bins1 = [425,1000,3000,5000]

# Create labels for these bins
group_labels1 = ["Small(<1000)", "Medium(1000-2500)", "Large(2500-5000)"]

schoolsize_series = pd.cut(reorg_schoolsummary["Total Students"], bins1, labels=group_labels1)
schoolsize_series

School Name
Huang High School        Medium(1000-2500)
Figueroa High School     Medium(1000-2500)
Shelton High School      Medium(1000-2500)
Hernandez High School     Large(2500-5000)
Griffin High School      Medium(1000-2500)
Wilson High School       Medium(1000-2500)
Cabrera High School      Medium(1000-2500)
Bailey High School        Large(2500-5000)
Holden High School            Small(<1000)
Pena High School              Small(<1000)
Wright High School       Medium(1000-2500)
Rodriguez High School     Large(2500-5000)
Johnson High School       Large(2500-5000)
Ford High School         Medium(1000-2500)
Thomas High School       Medium(1000-2500)
Name: Total Students, dtype: category
Categories (3, object): [Small(<1000) < Medium(1000-2500) < Large(2500-5000)]

In [222]:
reorg_schoolsummary3 = reorg_schoolsummary
reorg_schoolsummary3["School Size"] = schoolsize_series
reorg_schoolsummary3

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending ranges (Per Student),School Size
School Name,,,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,77.72,94.48,86.10,$635-$655,Medium(1000-2500)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,77.18,94.54,85.86,$635-$655,Medium(1000-2500)
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,100.00,100.00,100.00,$595-$615,Medium(1000-2500)
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,77.73,94.61,86.17,$635-$655,Large(2500-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,100.00,100.00,100.00,$615-$635,Medium(1000-2500)
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,100.00,100.00,100.00,<$595,Medium(1000-2500)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,100.00,100.00,100.00,<$595,Medium(1000-2500)
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,77.91,94.55,86.23,$615-$635,Large(2500-5000)
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,100.00,100.00,100.00,<$595,Small(<1000)


In [224]:
schoolsize_grp = reorg_schoolsummary3.groupby("School Size")
#perStudent_grp.mean()
schoolsize_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,100.000000,100.000000,100.000000
Medium(1000-2500),81.176821,82.933187,92.566667,98.098889,95.333333
Large(2500-5000),77.063340,80.919864,77.887500,94.565000,86.225000


In [225]:
schooltype_grp = reorg_schoolsummary.groupby("School Type")
schooltype_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,100.000000,100.00,100.000000
District,76.956733,80.966636,77.807143,94.45,86.128571
